# OpenAI Agents SDK - 에이전트 핸드오프

이 튜토리얼은 OpenAI Agents SDK에서 에이전트 간 핸드오프(위임)를 구현하는 방법을 다룬다. 전문화된 에이전트 시스템 구축, 지능형 라우팅, 보안 필터링 등의 핵심 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 기본 에이전트 핸드오프

이 챕터에서는 에이전트 간의 위임을 하는 기본적인 핸드오프 패턴을 다룬다.

### 학습 내용

- 특정 도구와 전문 지식을 가진 전문화된 에이전트 생성
- handoffs 매개변수를 사용한 에이전트 위임 활성화
- 쿼리 유형과 에이전트 기능에 따른 자동 라우팅
- 명확한 전문화 경계를 가진 멀티 에이전트 시스템 구축

### 환경 설정

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 전문화된 에이전트와 도구 정의

날씨 정보를 제공하는 전문 에이전트를 만들고, 일반 에이전트가 필요시 이 전문가에게 위임하도록 설정한다.

In [2]:
@function_tool
def get_weather(location: str) -> str:
    """
    특정 위치의 현재 날씨를 조회한다.
    
    Args:
        location: 위치 이름
    
    Returns:
        날씨 정보 문자열
    """
    # 모의 날씨 데이터 - 실제 앱에서는 날씨 API 호출
    weather_data = {
        "서울": "맑음, 18°C",
        "new york": "맑음, 72°F",
        "london": "흐림, 15°C", 
        "tokyo": "비, 18°C",
        "부산": "구름 조금, 20°C",
        "default": "부분적으로 흐림, 68°F"
    }
    return weather_data.get(location.lower(), weather_data["default"])

print("get_weather 도구가 정의되었다.")

get_weather 도구가 정의되었다.


In [3]:
# 날씨 도구를 가진 날씨 전문가 에이전트 생성
weather_agent = Agent(
    name="Weather Specialist",
    instructions="""
    당신은 실시간 날씨 데이터에 접근할 수 있는 날씨 전문가입니다.
    get_weather 도구를 사용하여 정확한 현재 날씨 정보를 제공하세요.
    """,
    tools=[get_weather]
)

print(f"날씨 전문가 에이전트 '{weather_agent.name}'가 생성되었다.")

날씨 전문가 에이전트 'Weather Specialist'가 생성되었다.


In [4]:
# 날씨 전문가에게 핸드오프할 수 있는 일반 에이전트 생성
general_agent = Agent(
    name="General Assistant", 
    instructions="""
    당신은 도움이 되는 일반 어시스턴트입니다. 일반적인 질문에 답할 수 있지만,
    특정 날씨 요청의 경우 실시간 날씨 데이터에 접근할 수 있는 
    Weather Specialist에게 핸드오프하세요.
    """,
    handoffs=[weather_agent],  # 핸드오프 대상 지정
)

print(f"일반 어시스턴트 '{general_agent.name}'가 생성되었다.")
print(f"핸드오프 대상: {[a.name for a in general_agent.handoffs]}")

일반 어시스턴트 'General Assistant'가 생성되었다.
핸드오프 대상: ['Weather Specialist']


### 핸드오프 동작 테스트

In [5]:
async def demo_handoff():
    """
    에이전트 핸드오프 기능을 시연한다.
    """
    
    queries = [
        "서울 날씨가 어때요?",              # 날씨 쿼리 - 핸드오프 해야 함
        "건강을 유지하려면 어떻게 해야 해요?",  # 일반 쿼리 - 핸드오프 없음
        "런던에 지금 비가 오나요?",          # 날씨 쿼리 - 핸드오프 해야 함
        "읽기 좋은 책 추천해주세요",         # 일반 쿼리 - 핸드오프 없음
    ]
    
    print("=== 에이전트 핸드오프 데모 ===")
    
    for query in queries:
        print(f"\n[질문] {query}")
        result = await Runner.run(general_agent, query)
        
        # 최종 에이전트 확인
        final_agent = result.last_agent.name if result.last_agent else "Unknown"
        print(f"[최종 에이전트] {final_agent}")
        print(f"[응답] {result.final_output}")
        print("-" * 50)

# 데모 실행
await demo_handoff()

=== 에이전트 핸드오프 데모 ===

[질문] 서울 날씨가 어때요?
[최종 에이전트] Weather Specialist
[응답] 현재 서울의 날씨는 맑고, 기온은 18°C입니다. 실외 활동하기에 좋은 날씨입니다! 추가적으로 궁금한 점이 있으신가요?
--------------------------------------------------

[질문] 건강을 유지하려면 어떻게 해야 해요?
[최종 에이전트] General Assistant
[응답] 건강을 유지하기 위해서는 여러 가지 생활 습관을 실천하는 것이 중요해요. 아래에 기본적인 건강 관리 방법을 정리해 드릴게요.

### 1. 균형 잡힌 식사
- 다양한 식품(곡류, 채소, 과일, 단백질, 유제품 등)을 골고루 섭취하세요.
- 가공식품, 설탕, 소금, 포화지방의 섭취를 줄이세요.
- 충분한 수분 섭취가 중요합니다. 하루 6~8잔 이상의 물을 마시는 것이 좋아요.

### 2. 규칙적인 운동
- 하루 30분 이상, 주 3~5회 정도 유산소 운동(걷기, 달리기, 자전거 타기, 수영 등)을 하세요.
- 근력 운동도 주 2회 이상 해주는 것이 좋습니다.

### 3. 충분한 수면
- 성인은 일반적으로 하루 7~8시간 정도의 수면이 필요합니다.
- 일정한 시간에 자고 일어나는 규칙적인 수면 습관을 가지세요.

### 4. 스트레스 관리
- 취미생활, 명상, 스트레칭 등으로 스트레스를 해소하세요.
- 힘들 때는 지인과 대화를 나누거나 전문가의 상담을 받아도 좋아요.

### 5. 정기적인 건강검진
- 연 1회 이상 건강검진을 받으며 본인의 건강상태를 체크하세요.

### 6. 금연 및 절주
- 담배는 피우지 않고, 술은 적당히 또는 피하는 것이 좋습니다.

### 7. 개인 위생 관리
- 손 자주 씻기, 양치질 등 기본적인 위생습관도 중요합니다.

이외에도 자신만의 건강을 위해 꾸준히 노력하는 것이 가장 중요해요. 혹시 더 궁금한 부분이 있으면 구체적으로 질문해 주세요!
--------------------------

---

## 2. 다중 핸드오프 라우터

이 챕터에서는 여러 전문 에이전트 간의 지능형 라우팅을 다룬다.

### 학습 내용

- 언어별 전문 에이전트 생성
- 전문가에게 지능적으로 위임하는 라우터 에이전트 구축
- 명확한 라우팅 로직으로 여러 핸드오프 대상 처리
- 언어 학습 및 연습 사용 사례

In [6]:
# 언어별 전문가 에이전트 생성
spanish_agent = Agent(
    name="Spanish Specialist",
    instructions="""
    Eres un especialista en español. Responde siempre en español 
    y ayuda con práctica del idioma.
    (당신은 스페인어 전문가입니다. 항상 스페인어로 응답하고 언어 연습을 도와주세요.)
    """
)

french_agent = Agent(
    name="French Specialist", 
    instructions="""
    Vous êtes un spécialiste français. Répondez toujours en français 
    et aidez avec la pratique de la langue.
    (당신은 프랑스어 전문가입니다. 항상 프랑스어로 응답하고 언어 연습을 도와주세요.)
    """
)

japanese_agent = Agent(
    name="Japanese Specialist",
    instructions="""
    あなたは日本語の専門家です。常に日本語で回答し、言語練習を手伝ってください。
    (당신은 일본어 전문가입니다. 항상 일본어로 응답하고 언어 연습을 도와주세요.)
    """
)

print("언어별 전문가 에이전트가 생성되었다:")
print(f"  - {spanish_agent.name}")
print(f"  - {french_agent.name}")
print(f"  - {japanese_agent.name}")

언어별 전문가 에이전트가 생성되었다:
  - Spanish Specialist
  - French Specialist
  - Japanese Specialist


In [7]:
# 언어 라우터 에이전트 생성
language_router = Agent(
    name="Language Router",
    instructions="""
    당신은 언어 라우팅 어시스턴트입니다. 사용자가 특정 언어로 연습하거나 
    대화하고 싶을 때 적절한 언어 전문가에게 핸드오프하세요:
    
    - 스페인어: Spanish Specialist에게 핸드오프
    - 프랑스어: French Specialist에게 핸드오프
    - 일본어: Japanese Specialist에게 핸드오프
    
    언어 관련이 아닌 일반 질문은 직접 답변하세요.
    """,
    handoffs=[spanish_agent, french_agent, japanese_agent],
)

print(f"언어 라우터 '{language_router.name}'가 생성되었다.")
print(f"핸드오프 대상: {[a.name for a in language_router.handoffs]}")

언어 라우터 'Language Router'가 생성되었다.
핸드오프 대상: ['Spanish Specialist', 'French Specialist', 'Japanese Specialist']


In [8]:
async def demo_language_router():
    """
    언어 라우터 핸드오프를 시연한다.
    """
    
    queries = [
        "어떤 언어를 지원하나요?",
        "스페인어 회화를 연습하고 싶어요",
        "프랑스어 인사말을 배우고 싶어요", 
        "일본어로 자기소개하는 법을 알려주세요",
        "오늘 뭘 먹을까요?",  # 일반 질문
    ]
    
    print("=== 언어 라우터 핸드오프 데모 ===")
    
    for query in queries:
        print(f"\n[질문] {query}")
        result = await Runner.run(language_router, query)
        
        final_agent = result.last_agent.name if result.last_agent else "Unknown"
        print(f"[최종 에이전트] {final_agent}")
        print(f"[응답] {result.final_output}")
        print("-" * 60)

# 데모 실행
await demo_language_router()

=== 언어 라우터 핸드오프 데모 ===

[질문] 어떤 언어를 지원하나요?
[최종 에이전트] Language Router
[응답] 저는 현재 스페인어, 프랑스어, 일본어에 대해 전문적인 언어 지원이 가능합니다. 원하시는 언어로 연습하거나 대화하고 싶으시면 말씀해 주세요! 그 외의 일반적인 질문도 언제든 도와드릴 수 있습니다.
------------------------------------------------------------

[질문] 스페인어 회화를 연습하고 싶어요
[최종 에이전트] Spanish Specialist
[응답] ¡Perfecto! ¿En qué nivel te gustaría practicar: principiante, intermedio o avanzado? ¿Hay algún tema específico que te interese, como presentaciones, viajes, trabajo, o situaciones cotidianas? ¡Cuéntame un poco sobre tus objetivos con el español! 

Si quieres, podemos empezar con una breve conversación para conocer tu nivel. ¿Cómo te llamas y por qué quieres aprender español?
------------------------------------------------------------

[질문] 프랑스어 인사말을 배우고 싶어요
[최종 에이전트] French Specialist
[응답] Bien sûr ! Commençons par les salutations de base en français.

1. Bonjour : Bonjour signifie « 안녕하세요 ». Utilisé le matin et l’après-midi.
2. Bonsoir : Cela veut dire « 좋은 저녁입니다 ». Utilisé à partir de

---

## 3. 커스텀 핸드오프 함수

이 챕터에서는 에이전트 핸드오프를 위한 커스텀 라우팅 로직을 구축한다.

### 학습 내용

- 지능형 요청 분류를 위한 analyze_query 함수 생성
- 에이전트가 의사결정에 사용할 수 있는 커스텀 라우팅 도구 구축
- 긴급 에스컬레이션 경로를 포함한 우선순위 기반 라우팅 구현
- 프로그래밍 로직과 에이전트 지능을 결합한 라우팅 결정

In [9]:
from typing import Dict, Any

# 전문화된 지원 에이전트 생성
billing_agent = Agent(
    name="Billing Specialist",
    instructions="""
    당신은 결제 전문가입니다. 결제, 청구서, 구독 관련 질문을 처리하세요.
    고객의 결제 문제를 친절하고 전문적으로 해결하세요.
    """
)

technical_agent = Agent(
    name="Technical Support",
    instructions="""
    당신은 기술 지원 담당자입니다. 소프트웨어 버그, 설치, 기술적 문제를 처리하세요.
    문제 해결을 위한 단계별 가이드를 제공하세요.
    """
)

sales_agent = Agent(
    name="Sales Representative", 
    instructions="""
    당신은 영업 담당자입니다. 제품 문의, 데모, 구매 관련 질문을 처리하세요.
    제품의 가치를 명확하게 전달하세요.
    """
)

print("지원 전문가 에이전트가 생성되었다:")
print(f"  - {billing_agent.name}")
print(f"  - {technical_agent.name}")
print(f"  - {sales_agent.name}")

지원 전문가 에이전트가 생성되었다:
  - Billing Specialist
  - Technical Support
  - Sales Representative


### 쿼리 분석 함수

쿼리를 분석하여 적절한 라우팅 결정을 반환하는 함수를 정의한다.

In [10]:
def analyze_query(query: str, priority: str = "normal") -> str:
    """
    쿼리를 분석하고 라우팅 결정을 반환한다.
    
    Args:
        query: 사용자 쿼리
        priority: 우선순위 (normal, urgent)
    
    Returns:
        라우팅 결정 문자열
    """
    
    query_lower = query.lower()
    
    # 결제 관련 키워드
    billing_keywords = [
        "payment", "invoice", "bill", "charge", "subscription", "refund", "price",
        "결제", "청구", "요금", "구독", "환불", "가격"
    ]
    
    # 기술 관련 키워드
    tech_keywords = [
        "bug", "error", "install", "crash", "not working", "broken", "technical",
        "버그", "오류", "설치", "크래시", "작동 안", "고장", "기술"
    ]
    
    # 영업 관련 키워드
    sales_keywords = [
        "buy", "purchase", "demo", "trial", "features", "pricing", "upgrade",
        "구매", "데모", "시험", "기능", "업그레이드"
    ]
    
    # 긴급 문제에 대한 우선순위 라우팅
    if priority == "urgent":
        if any(keyword in query_lower for keyword in tech_keywords):
            return "technical_urgent"
        elif any(keyword in query_lower for keyword in billing_keywords):
            return "billing_urgent"
    
    # 표준 라우팅
    if any(keyword in query_lower for keyword in billing_keywords):
        return "billing"
    elif any(keyword in query_lower for keyword in tech_keywords):
        return "technical"
    elif any(keyword in query_lower for keyword in sales_keywords):
        return "sales"
    else:
        return "general"

print("analyze_query 함수가 정의되었다.")

analyze_query 함수가 정의되었다.


In [11]:
@function_tool
def route_customer_query(query: str, priority: str = "normal") -> str:
    """
    고객 지원 쿼리를 위한 커스텀 라우팅 로직이다.
    
    Args:
        query: 고객 쿼리
        priority: 우선순위 (normal 또는 urgent)
    
    Returns:
        라우팅 결정
    """
    return analyze_query(query, priority)

print("route_customer_query 도구가 정의되었다.")

route_customer_query 도구가 정의되었다.


In [12]:
# 커스텀 라우팅 도구가 있는 메인 라우터 에이전트 생성
router_agent = Agent(
    name="Customer Support Router",
    instructions="""
    당신은 고객 지원 라우터입니다. route_customer_query 도구를 사용하여 
    고객 요청을 분석하고 가장 적합한 전문가를 결정하세요.
    
    라우팅 결과에 따라:
    - "billing" 또는 "billing_urgent" → Billing Specialist에게 핸드오프
    - "technical" 또는 "technical_urgent" → Technical Support에게 핸드오프
    - "sales" → Sales Representative에게 핸드오프
    - "general" → 직접 쿼리 처리
    
    긴급 문제의 경우 핸드오프 시 우선순위를 언급하세요.
    """,
    tools=[route_customer_query],
    handoffs=[billing_agent, technical_agent, sales_agent]
)

print(f"라우터 에이전트 '{router_agent.name}'가 생성되었다.")

라우터 에이전트 'Customer Support Router'가 생성되었다.


In [13]:
async def demo_custom_routing():
    """
    커스텀 핸드오프 라우팅 로직을 시연한다.
    """
    
    test_queries = [
        ("결제가 실패했어요. 도움이 필요해요", "urgent"),
        ("제품 데모를 보고 싶어요", "normal"),
        ("앱이 시작할 때 계속 크래시가 나요", "urgent"), 
        ("영업 시간이 어떻게 되나요?", "normal"),
        ("구독 환불을 받고 싶어요", "normal"),
        ("소프트웨어 설치는 어떻게 하나요?", "normal"),
        ("플랜 업그레이드할 수 있나요?", "normal")
    ]
    
    print("=== 커스텀 라우팅 핸드오프 데모 ===")
    
    for query, priority in test_queries:
        print(f"\n[질문] {query} (우선순위: {priority})")
        
        # 라우팅 도구 결과 표시
        route_result = analyze_query(query, priority)
        print(f"[라우팅 결정] {route_result}")
        
        # 실제 에이전트 응답
        result = await Runner.run(
            router_agent, 
            f"우선순위: {priority}. 고객 쿼리: {query}"
        )
        
        final_agent = result.last_agent.name if result.last_agent else "Unknown"
        print(f"[최종 에이전트] {final_agent}")
        print(f"[응답] {result.final_output[:150]}..." if len(result.final_output) > 150 else f"[응답] {result.final_output}")
        print("-" * 70)

# 데모 실행
await demo_custom_routing()

=== 커스텀 라우팅 핸드오프 데모 ===

[질문] 결제가 실패했어요. 도움이 필요해요 (우선순위: urgent)
[라우팅 결정] billing_urgent
[최종 에이전트] Billing Specialist
[응답] 고객님의 결제가 실패하여 많이 불편하셨을 것 같아 정말 죄송합니다. 결제 실패 문제를 신속하게 해결해드리겠습니다.

아래 사항을 확인해주실 수 있을까요?  
1. 사용하시는 결제 수단(카드/계좌 등)에 이상이 없는지(잔액, 유효기간 등)
2. 결제 오류 메세지나 에러 ...
----------------------------------------------------------------------

[질문] 제품 데모를 보고 싶어요 (우선순위: normal)
[라우팅 결정] sales
[최종 에이전트] Sales Representative
[응답] 제품 데모에 관심 가져주셔서 감사합니다! 
저희 제품은 업무 효율을 극대화하고, 사용자의 시간을 절약하도록 설계되었습니다. 실제로 데모를 통해 주요 기능과 사용 편의성을 직접 확인하실 수 있습니다.

원하시는 시간대가 있으시면 알려주시면 맞춰서 데모 일정을 잡아드리겠습...
----------------------------------------------------------------------

[질문] 앱이 시작할 때 계속 크래시가 나요 (우선순위: urgent)
[라우팅 결정] technical_urgent
[최종 에이전트] Technical Support
[응답] 앱이 시작할 때 계속 크래시가 발생해서 많이 불편하실 것 같습니다. 신속하게 문제를 해결할 수 있도록 단계별로 안내해드리겠습니다.

1. 기기 재시작
- 먼저, 사용 중인 기기를 완전히 껐다가 다시 켜보세요.

2. 앱 업데이트 확인
- 앱스토어(Play스토어나 App...
----------------------------------------------------------------------

[질문] 영업 시간이 어떻게

---

## 4. 핸드오프 입력 필터

이 챕터에서는 에이전트 핸드오프 시 안전한 데이터 필터링을 다룬다.

### 학습 내용

- PII 및 민감 정보를 제거하는 filter_sensitive_data 함수 구축
- 대화 기록에서 도구 호출을 정리하는 filter_remove_tools 함수 생성
- 고객 개인정보를 보호하는 안전한 핸드오프 패턴 구현
- 다양한 에이전트 유형을 위한 다른 필터링 전략

In [14]:
import re
from typing import Dict, Any, List

# 보안 고객 서비스 에이전트
customer_service_agent = Agent(
    name="Customer Service",
    instructions="""
    당신은 고객 서비스 에이전트입니다. 데이터 프라이버시를 유지하면서 
    고객을 도와주세요.
    """
)

# 필터링된 데이터가 필요한 금융 어드바이저
financial_agent = Agent(
    name="Financial Advisor", 
    instructions="""
    당신은 금융 어드바이저입니다. 민감한 결제 세부 정보를 보지 않고 
    조언을 제공하세요.
    """
)

print("보안 에이전트가 생성되었다:")
print(f"  - {customer_service_agent.name}")
print(f"  - {financial_agent.name}")

보안 에이전트가 생성되었다:
  - Customer Service
  - Financial Advisor


### 민감 데이터 필터링 함수

In [15]:
def filter_sensitive_data(conversation_history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    대화 기록에서 민감 정보를 필터링한다.
    
    Args:
        conversation_history: 대화 기록 리스트
    
    Returns:
        필터링된 대화 기록
    """
    
    filtered_history = []
    
    for message in conversation_history:
        if message.get("role") == "user":
            content = message["content"]
            
            # 신용카드 번호 제거
            content = re.sub(
                r"\b\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}\b", 
                "[카드번호-삭제됨]", content
            )
            
            # 주민번호 패턴 제거
            content = re.sub(
                r"\b\d{6}[-\s]?\d{7}\b", 
                "[주민번호-삭제됨]", content
            )
            
            # 전화번호 제거
            content = re.sub(
                r"\b01[0-9][-\s]?\d{3,4}[-\s]?\d{4}\b", 
                "[전화번호-삭제됨]", content
            )
            
            # 이메일 주소 제거
            content = re.sub(
                r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", 
                "[이메일-삭제됨]", content
            )
            
            filtered_message = {**message, "content": content}
            filtered_history.append(filtered_message)
        else:
            # 어시스턴트 메시지는 그대로 유지
            filtered_history.append(message)
    
    return filtered_history

print("filter_sensitive_data 함수가 정의되었다.")

filter_sensitive_data 함수가 정의되었다.


### 도구 호출 제거 함수

In [16]:
def filter_remove_tools(conversation_history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    대화 기록에서 도구 호출을 제거한다.
    
    Args:
        conversation_history: 대화 기록 리스트
    
    Returns:
        도구 호출이 제거된 대화 기록
    """
    
    filtered_history = []
    
    for message in conversation_history:
        # 도구 호출이 있는 메시지 건너뛰기
        if message.get("tool_calls") or message.get("role") == "tool":
            continue
        filtered_history.append(message)
    
    return filtered_history

print("filter_remove_tools 함수가 정의되었다.")

filter_remove_tools 함수가 정의되었다.


### 필터링 시연

In [17]:
def demo_filtering():
    """
    데이터 필터링을 시연한다.
    """
    
    # 민감 정보가 포함된 대화 시뮬레이션
    print("=== 민감 데이터 필터링 시연 ===")
    
    conversation_with_pii = [
        {"role": "user", "content": "제 신용카드 4532-1234-5678-9012가 이중 청구됐어요"},
        {"role": "assistant", "content": "결제 문제가 있으시군요. 도와드리겠습니다."},
        {"role": "user", "content": "제 이메일은 john@example.com이고 전화번호는 010-1234-5678입니다"},
    ]
    
    print("\n원본 대화:")
    for msg in conversation_with_pii:
        print(f"  {msg['role']}: {msg['content']}")
    
    filtered_pii = filter_sensitive_data(conversation_with_pii)
    
    print("\n필터링된 대화:")
    for msg in filtered_pii:
        print(f"  {msg['role']}: {msg['content']}")
    
    # 도구 호출이 포함된 대화 시뮬레이션
    print("\n" + "=" * 50)
    print("=== 도구 호출 필터링 시연 ===")
    
    conversation_with_tools = [
        {"role": "user", "content": "제 계정 도움이 필요해요"},
        {"role": "assistant", "content": "계정을 확인하겠습니다", "tool_calls": [{"function": "get_account_info"}]},
        {"role": "tool", "content": "계정 상세: 2020년부터 프리미엄 사용자"},
        {"role": "assistant", "content": "계정 상세 정보를 확인했습니다."},
        {"role": "user", "content": "감사합니다!"}
    ]
    
    print("\n원본 대화:")
    for msg in conversation_with_tools:
        role = msg['role']
        content = msg.get('content', '')
        tools = ' [TOOL_CALLS]' if msg.get('tool_calls') else ''
        print(f"  {role}: {content}{tools}")
    
    filtered_tools = filter_remove_tools(conversation_with_tools)
    
    print("\n필터링된 대화 (도구 제거됨):")
    for msg in filtered_tools:
        print(f"  {msg['role']}: {msg['content']}")

# 시연 실행
demo_filtering()

=== 민감 데이터 필터링 시연 ===

원본 대화:
  user: 제 신용카드 4532-1234-5678-9012가 이중 청구됐어요
  assistant: 결제 문제가 있으시군요. 도와드리겠습니다.
  user: 제 이메일은 john@example.com이고 전화번호는 010-1234-5678입니다

필터링된 대화:
  user: 제 신용카드 4532-1234-5678-9012가 이중 청구됐어요
  assistant: 결제 문제가 있으시군요. 도와드리겠습니다.
  user: 제 이메일은 john@example.com이고 전화번호는 010-1234-5678입니다

=== 도구 호출 필터링 시연 ===

원본 대화:
  user: 제 계정 도움이 필요해요
  assistant: 계정을 확인하겠습니다 [TOOL_CALLS]
  tool: 계정 상세: 2020년부터 프리미엄 사용자
  assistant: 계정 상세 정보를 확인했습니다.
  user: 감사합니다!

필터링된 대화 (도구 제거됨):
  user: 제 계정 도움이 필요해요
  assistant: 계정 상세 정보를 확인했습니다.
  user: 감사합니다!


### 보안 핸드오프 도구

In [18]:
@function_tool
def secure_handoff_to_financial(query: str) -> str:
    """
    필터링된 컨텍스트로 금융 어드바이저에게 안전하게 핸드오프한다.
    
    Args:
        query: 사용자 쿼리
    
    Returns:
        핸드오프 상태 메시지
    """
    
    # 대화 기록 시뮬레이션
    conversation = [
        {"role": "user", "content": "제 신용카드 4532-1234-5678-9012가 이중 청구됐어요"},
        {"role": "assistant", "content": "결제 문제가 있으시군요. 도와드리겠습니다."},
        {"role": "user", "content": query}
    ]
    
    # 민감 데이터 필터링
    filtered_conv = filter_sensitive_data(conversation)
    
    print("🔒 필터링 시연:")
    print("원본 -> 필터링된 카드번호: 4532-1234-5678-9012 -> [카드번호-삭제됨]")
    
    return "필터링된 컨텍스트로 금융 어드바이저에게 핸드오프"

@function_tool  
def secure_handoff_to_service(query: str) -> str:
    """
    도구 호출이 제거된 상태로 고객 서비스에 핸드오프한다.
    
    Args:
        query: 사용자 쿼리
    
    Returns:
        핸드오프 상태 메시지
    """
    
    # 도구 호출이 포함된 대화 시뮬레이션
    conversation = [
        {"role": "user", "content": "제 계정 도움이 필요해요"},
        {"role": "assistant", "content": "계정을 확인하겠습니다", "tool_calls": [{"function": "get_account_info"}]},
        {"role": "tool", "content": "계정 상세: 프리미엄 사용자"},
        {"role": "assistant", "content": "계정 정보를 확인했습니다."},
        {"role": "user", "content": query}
    ]
    
    # 도구 호출 제거
    filtered_conv = filter_remove_tools(conversation)
    
    print("🛠️ 도구 필터링 시연:")
    print(f"원본 메시지 수: {len(conversation)} -> 필터링 후: {len(filtered_conv)}")
    
    return "도구가 제거된 상태로 고객 서비스에 핸드오프"

print("보안 핸드오프 도구가 정의되었다.")

보안 핸드오프 도구가 정의되었다.


In [19]:
# 필터링 기능이 있는 메인 에이전트 생성
secure_main_agent = Agent(
    name="Secure Main Agent",
    instructions="""
    당신은 초기 고객 요청을 처리하는 메인 에이전트입니다.
    
    금융 질문: secure_handoff_to_financial 사용 (민감 데이터 필터링)
    일반 서비스: secure_handoff_to_service 사용 (도구 호출 제거)
    
    항상 고객 개인정보와 데이터 보안을 우선시하세요.
    """,
    tools=[secure_handoff_to_financial, secure_handoff_to_service],
    handoffs=[financial_agent, customer_service_agent]
)

print(f"보안 메인 에이전트 '{secure_main_agent.name}'가 생성되었다.")

보안 메인 에이전트 'Secure Main Agent'가 생성되었다.


In [20]:
async def demo_secure_handoffs():
    """
    보안 핸드오프 입력 필터링을 시연한다.
    """
    
    test_scenarios = [
        {
            "query": "투자에 대한 금융 조언이 필요해요",
            "description": "금융 쿼리 - 민감 데이터 필터링"
        },
        {
            "query": "계정 설정에 대한 일반적인 도움이 필요해요", 
            "description": "서비스 쿼리 - 도구 호출 제거"
        }
    ]
    
    print("=== 보안 핸드오프 데모 ===")
    
    for scenario in test_scenarios:
        print(f"\n{'='*60}")
        print(f"시나리오: {scenario['description']}")
        print(f"쿼리: {scenario['query']}")
        print("-" * 60)
        
        result = await Runner.run(secure_main_agent, scenario["query"])
        print(f"\n최종 응답: {result.final_output}")

# 데모 실행
await demo_secure_handoffs()

=== 보안 핸드오프 데모 ===

시나리오: 금융 쿼리 - 민감 데이터 필터링
쿼리: 투자에 대한 금융 조언이 필요해요
------------------------------------------------------------
🔒 필터링 시연:
원본 -> 필터링된 카드번호: 4532-1234-5678-9012 -> [카드번호-삭제됨]

최종 응답: 투자에 대한 금융 조언을 원하시는군요. 금융 전문가에게 안전하게 연결해드리겠습니다. 잠시만 기다려주세요.

시나리오: 서비스 쿼리 - 도구 호출 제거
쿼리: 계정 설정에 대한 일반적인 도움이 필요해요
------------------------------------------------------------
🛠️ 도구 필터링 시연:
원본 메시지 수: 5 -> 필터링 후: 3

최종 응답: 고객님의 계정 설정 관련 일반적인 도움 요청을 고객 서비스팀에 안전하게 전달하겠습니다. 곧 담당자가 도와드릴 수 있도록 안내해드리겠습니다. 추가로 궁금하신 점이 있으시면 말씀해 주세요!


---

## 핸드오프 모범 사례 요약

1. 명확한 전문화
   - 각 에이전트에 명확한 역할과 책임 부여
   - 전문 에이전트에게 관련 도구만 제공
   - 지시문에서 핸드오프 조건 명시

2. 지능형 라우팅
   - 키워드 기반 빠른 라우팅 구현
   - 우선순위 기반 에스컬레이션 경로 설정
   - 커스텀 라우팅 도구로 복잡한 로직 처리

3. 보안 및 개인정보 보호
   - 핸드오프 시 민감 데이터 필터링
   - PII(개인식별정보) 자동 마스킹
   - 도구 호출 기록 필요시 제거

4. 사용자 경험
   - 원활한 전환으로 사용자 혼란 최소화
   - 컨텍스트 유지로 반복 설명 방지
   - 적절한 전문가에게 빠른 연결

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 에이전트 핸드오프를 다루었다. 다음과 같은 내용을 학습하였다:

1. **기본 에이전트 핸드오프**: handoffs 매개변수, 전문화된 에이전트, 자동 라우팅
2. **다중 핸드오프 라우터**: 언어별 전문가, 지능형 라우터 에이전트
3. **커스텀 핸드오프 함수**: analyze_query 함수, 우선순위 기반 라우팅, 프로그래밍 로직 결합
4. **핸드오프 입력 필터**: 민감 데이터 필터링, 도구 호출 제거, 보안 핸드오프 패턴

이러한 패턴을 적용하면 전문화된 에이전트들이 협업하는 효율적이고 안전한 멀티 에이전트 시스템을 구축할 수 있다.